In [1]:
import pandas as pd 
import pymongo
import dns
from pymongo import MongoClient
import os

In [14]:
# Start with employment data

# Create instance of MongoClient
client = MongoClient()
# Connection URI
connStr = os.getenv('MONGO_CONN')
client = MongoClient(connStr)
# Select database
db = client['MSA']
# Select the collection within the database
data = db.Employment_clean
# Convert entire collection to Pandas dataframe
df_emp_total = pd.DataFrame(list(data.find()))
df_emp_total.head()

,_id,CBSA,MSA,year,month,value
0,603818923390f928b3bdb22b,11500,"Anniston-Oxford, AL",2019,December,44930
1,603818923390f928b3bdb22c,11500,"Anniston-Oxford, AL",2019,November,45076
2,603818923390f928b3bdb22d,11500,"Anniston-Oxford, AL",2019,October,45127
3,603818923390f928b3bdb22e,11500,"Anniston-Oxford, AL",2019,September,44455
4,603818923390f928b3bdb22f,11500,"Anniston-Oxford, AL",2019,August,44535


In [15]:
df_emp_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46080 entries, 0 to 46079
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   _id     46080 non-null  object
 1   CBSA    46080 non-null  int64 
 2   MSA     46080 non-null  object
 3   year    46080 non-null  int64 
 4   month   46080 non-null  object
 5   value   46080 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 2.1+ MB


In [16]:
df_emp_total = df_emp_total[df_emp_total['month'] == 'December']
df_emp_total

,_id,CBSA,MSA,year,month,value
0,603818923390f928b3bdb22b,11500,"Anniston-Oxford, AL",2019,December,44930
12,603818923390f928b3bdb237,11500,"Anniston-Oxford, AL",2018,December,44317
24,603818923390f928b3bdb243,11500,"Anniston-Oxford, AL",2017,December,43746
36,603818923390f928b3bdb24f,11500,"Anniston-Oxford, AL",2016,December,43310
48,603818923390f928b3bdb25b,11500,"Anniston-Oxford, AL",2015,December,42421
...,...,...,...,...,...,...
46020,603818923390f928b3be65ef,49340,"Worcester, MA-CT",2014,December,329934
46032,603818923390f928b3be65fb,49340,"Worcester, MA-CT",2013,December,318688
46044,603818923390f928b3be6607,49340,"Worcester, MA-CT",2012,December,315202
46056,603818923390f928b3be6613,49340,"Worcester, MA-CT",2011,December,316265


In [18]:
df_emp_total.drop(columns=['month', 'MSA', '_id'], inplace=True)
df_emp_total.head()

,CBSA,year,value
0,11500,2019,44930
12,11500,2018,44317
24,11500,2017,43746
36,11500,2016,43310
48,11500,2015,42421


In [21]:
df_emp_total.sort_values(by='CBSA', inplace=True)
df_emp_total

,CBSA,year,value
36348,10180,2010,71455
36252,10180,2018,74942
36264,10180,2017,73630
36276,10180,2016,71786
36240,10180,2019,77346
...,...,...,...
2364,49740,2012,72913
2376,49740,2011,72344
2280,49740,2019,87383
2352,49740,2013,72013


In [28]:
df_emp_pivot = pd.pivot_table(df_emp_total, values = 'value', index=['CBSA'], columns = 'year').reset_index()
df_emp_pivot

year,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,10180,71455,72162,72210,72877,72614,71810,71786,73630,74942,77346
1,10420,333843,333177,333872,333060,341435,340759,340621,342135,344267,343479
2,10500,62624,63468,62213,61378,61231,62158,64476,64247,63869,64581
3,10540,49886,49832,48453,48629,50348,52528,54219,55353,56139,56468
4,10580,417173,415614,422290,422015,419228,426623,423587,422072,428744,433120
...,...,...,...,...,...,...,...,...,...,...,...
379,49420,100992,100872,100803,99456,103473,104395,108536,111456,114141,118684
380,49620,212564,214874,216426,216079,219070,222944,223961,224295,226766,228723
381,49660,238069,239089,237392,235580,237477,234932,230271,227637,227149,223658
382,49700,60165,61217,61259,61659,62462,64438,64836,67064,67306,72021


In [36]:
df_emp_pivot = df_emp_pivot.rename_axis(None, axis=1)
df_emp_pivot.head()

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,10180,71455,72162,72210,72877,72614,71810,71786,73630,74942,77346
1,10420,333843,333177,333872,333060,341435,340759,340621,342135,344267,343479
2,10500,62624,63468,62213,61378,61231,62158,64476,64247,63869,64581
3,10540,49886,49832,48453,48629,50348,52528,54219,55353,56139,56468
4,10580,417173,415614,422290,422015,419228,426623,423587,422072,428744,433120


In [44]:
col_list = df_emp_pivot.columns
new_col_list = []
for col in col_list:
    new_col_list.append(str(col))
df_emp_pivot.columns = new_col_list
df_emp_pivot.columns

Index(['CBSA', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019'],
      dtype='object')

In [45]:
# export employment clean dataframe back to mongo
employment_clean_collection = db.Employment_clean
df_dict = df_emp_pivot.to_dict(orient = 'records')
employment_clean_collection.insert_many(df_dict)

In [58]:
### Now lets put this together in a function
def clean_raw_data(df_raw):
    df_raw = df_raw[df_raw['month'] == 'December']
    # df_raw.drop(columns=['month', 'MSA', '_id'], inplace=True)
    df_raw.drop(columns=['month', '_id'], inplace=True)
    df_raw.sort_values(by='CBSA', inplace=True)
    df_pivot = pd.pivot_table(df_raw, values = 'value', index=['CBSA'], columns = 'year').reset_index()
    df_pivot = df_pivot.rename_axis(None, axis=1)
    col_list = df_pivot.columns
    new_col_list = []
    for col in col_list:
        new_col_list.append(str(col))
    df_pivot.columns = new_col_list
    return df_pivot

In [52]:
### Lets do labor force now

# Create instance of MongoClient
client = MongoClient()
# Connection URI
connStr = os.getenv('MONGO_CONN')
client = MongoClient(connStr)
# Select database
db = client['MSA']
# Select the collection within the database
data = db.LaborForce_clean
# Convert entire collection to Pandas dataframe
df_total = pd.DataFrame(list(data.find()))
df_total

,_id,CBSA,MSA,year,month,value
0,6037cfefd5c52ef30a33a881,11500,"Anniston-Oxford, AL",2019,December,46320
1,6037cfefd5c52ef30a33a882,11500,"Anniston-Oxford, AL",2019,November,46408
2,6037cfefd5c52ef30a33a883,11500,"Anniston-Oxford, AL",2019,October,46508
3,6037cfefd5c52ef30a33a884,11500,"Anniston-Oxford, AL",2019,September,45820
4,6037cfefd5c52ef30a33a885,11500,"Anniston-Oxford, AL",2019,August,46104
...,...,...,...,...,...,...
46075,6037cfefd5c52ef30a345c7c,49340,"Worcester, MA-CT",2010,May,339705
46076,6037cfefd5c52ef30a345c7d,49340,"Worcester, MA-CT",2010,April,339684
46077,6037cfefd5c52ef30a345c7e,49340,"Worcester, MA-CT",2010,March,339383
46078,6037cfefd5c52ef30a345c7f,49340,"Worcester, MA-CT",2010,February,339162


In [56]:
df_clean = clean_raw_data(df_total)
df_clean

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,10180,76486,76723,76209,76347,75197,74411,74615,76071,77323,79499
1,10420,369203,361648,360436,356986,359933,358167,359554,358853,360974,357787
2,10500,70825,71135,69101,67391,65957,66377,68471,67478,67082,66999
3,10540,56691,56016,54190,53221,54075,55841,56853,57888,58764,58446
4,10580,449154,447519,453433,445629,438706,444496,441435,440205,444430,449737
...,...,...,...,...,...,...,...,...,...,...,...
379,49420,114826,115596,114403,112408,115915,116743,119089,121256,124890,128962
380,49620,229344,231145,232525,228768,229489,232244,233206,232871,234409,237714
381,49660,265800,261511,259693,254507,251949,253137,245454,241154,240643,236372
382,49700,74371,74220,72725,71203,70521,71757,71897,72796,72979,77036


In [57]:
# export labor force clean dataframe back to mongo
clean_collection = db.LaborForce_clean
df_dict = df_clean.to_dict(orient = 'records')
clean_collection.insert_many(df_dict)

In [59]:
### Next we do UnemploymentRate

# Create instance of MongoClient
client = MongoClient()
# Connection URI
connStr = os.getenv('MONGO_CONN')
client = MongoClient(connStr)
# Select database
db = client['MSA']
# Select the collection within the database
data = db.UnemploymentRate_clean
# Convert entire collection to Pandas dataframe
df_total = pd.DataFrame(list(data.find()))
df_total

,_id,CBSA,year,month,value
0,6038baa2f3720aac8688bb68,11500,2019,December,3.0
1,6038baa2f3720aac8688bb69,11500,2019,November,2.9
2,6038baa2f3720aac8688bb6a,11500,2019,October,3.0
3,6038baa2f3720aac8688bb6b,11500,2019,September,3.0
4,6038baa2f3720aac8688bb6c,11500,2019,August,3.4
...,...,...,...,...,...
46075,6038baa3f3720aac86896f63,49340,2010,May,8.8
46076,6038baa3f3720aac86896f64,49340,2010,April,8.8
46077,6038baa3f3720aac86896f65,49340,2010,March,9.4
46078,6038baa3f3720aac86896f66,49340,2010,February,9.6


In [60]:
df_clean = clean_raw_data(df_total)
df_clean

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,10180,6.6,5.9,5.2,4.5,3.4,3.5,3.8,3.2,3.1,2.7
1,10420,9.6,7.9,7.4,6.7,5.1,4.9,5.3,4.7,4.6,4.0
2,10500,11.6,10.8,10.0,8.9,7.2,6.4,5.8,4.8,4.8,3.6
3,10540,12.0,11.0,10.6,8.6,6.9,5.9,4.6,4.4,4.5,3.4
4,10580,7.1,7.1,6.9,5.3,4.4,4.0,4.0,4.1,3.5,3.7
...,...,...,...,...,...,...,...,...,...,...,...
379,49420,12.0,12.7,11.9,11.5,10.7,10.6,8.9,8.1,8.6,8.0
380,49620,7.3,7.0,6.9,5.5,4.5,4.0,4.0,3.7,3.3,3.8
381,49660,10.4,8.6,8.6,7.4,5.7,7.2,6.2,5.6,5.6,5.4
382,49700,19.1,17.5,15.8,13.4,11.4,10.2,9.8,7.9,7.8,6.5


In [61]:
# export unemployment clean dataframe back to mongo
clean_collection = db.UnemploymentRate_clean
df_dict = df_clean.to_dict(orient = 'records')
clean_collection.insert_many(df_dict)

In [62]:
### Now clean up population

# Create instance of MongoClient
client = MongoClient()
# Connection URI
connStr = os.getenv('MONGO_CONN')
client = MongoClient(connStr)
# Select database
db = client['MSA']
# Select the collection within the database
data = db.population_clean
# Convert entire collection to Pandas dataframe
df_total = pd.DataFrame(list(data.find()))
df_total

,_id,CBSA,NAME,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,6031813991cc653aeedfe7c7,11540,"Appleton, WI",225948,227548,228656,229978,231627,233146,234200,235811,237011,237974
1,6031813991cc653aeedfe7c9,12020,"Athens-Clarke County, GA",193439,194375,195984,197590,198361,202565,205402,209106,211468,213750
2,6031813991cc653aeedfe7e0,14010,"Bloomington, IL",169810,170909,172721,175037,174105,173141,173070,172754,172410,171517
3,6031813991cc653aeedfe7ed,15540,"Burlington-South Burlington, VT",211538,213235,213962,214827,215676,217152,217447,218975,219939,220411
4,6031813991cc653aeedfe818,19660,"Deltona-Daytona Beach-Ormond Beach, FL",590506,591926,594969,600236,608604,621166,635859,647867,658186,668365
...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,6031813991cc653aeedfe8ca,38340,"Pittsfield, MA",131317,130547,130284,129488,128856,127858,127067,126353,125901,124944
380,6031813991cc653aeedfe8ef,41700,"San Antonio-New Braunfels, TX",2153009,2193923,2236981,2280002,2328894,2379232,2426276,2472121,2512379,2550960
381,6031813991cc653aeedfe8f3,42100,"Santa Cruz-Watsonville, CA",263147,264858,266340,268816,270861,273503,274396,274856,273841,273213
382,6031813991cc653aeedfe908,44180,"Springfield, MO",437349,440432,444329,448427,451556,455593,457917,462090,466273,470300


In [63]:
df_total.drop(columns=['_id', 'NAME'], inplace=True)

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,11540,225948,227548,228656,229978,231627,233146,234200,235811,237011,237974
1,12020,193439,194375,195984,197590,198361,202565,205402,209106,211468,213750
2,14010,169810,170909,172721,175037,174105,173141,173070,172754,172410,171517
3,15540,211538,213235,213962,214827,215676,217152,217447,218975,219939,220411
4,19660,590506,591926,594969,600236,608604,621166,635859,647867,658186,668365
...,...,...,...,...,...,...,...,...,...,...,...
379,38340,131317,130547,130284,129488,128856,127858,127067,126353,125901,124944
380,41700,2153009,2193923,2236981,2280002,2328894,2379232,2426276,2472121,2512379,2550960
381,42100,263147,264858,266340,268816,270861,273503,274396,274856,273841,273213
382,44180,437349,440432,444329,448427,451556,455593,457917,462090,466273,470300


In [67]:
df_total.sort_values(by='CBSA').reset_index(drop=True)

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,10180,165585,166634,167442,167473,168342,169688,170017,170429,171150,172060
1,10420,703031,703200,702109,703621,704908,704382,703524,703987,703855,703479
2,10500,154145,154545,153976,152667,151949,150387,149137,148090,147840,146726
3,10540,116891,118164,118273,118405,119042,120236,122769,125035,127451,129749
4,10580,871082,872778,874698,877065,878113,879085,879792,882158,882263,880381
...,...,...,...,...,...,...,...,...,...,...,...
379,49420,244256,245926,246127,246485,246870,247800,249327,249922,250562,250873
380,49620,435413,436701,437466,438932,440475,441741,443693,445484,447847,449058
381,49660,564826,562347,558530,555850,553039,548798,544582,541545,538226,536081
382,49700,167099,166971,166963,167597,168311,169327,170694,172782,173905,175639


In [68]:
# export unemployment clean dataframe back to mongo
clean_collection = db.Population_clean
df_dict = df_total.to_dict(orient = 'records')
clean_collection.insert_many(df_dict)

In [69]:
### Now the AnnualMeanWage

# Create instance of MongoClient
client = MongoClient()
# Connection URI
connStr = os.getenv('MONGO_CONN')
client = MongoClient(connStr)
# Select database
db = client['MSA']
# Select the collection within the database
data = db.OES_raw
# Convert entire collection to Pandas dataframe
df_total = pd.DataFrame(list(data.find()))
df_total

,_id,YEAR,AREA,CBSA,MSA_Name,state_code,PRIM_STATE,AREA_NAME,OCC_CODE,OCC_TITLE,...,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY,OCC_GROUP
0,6038df3058e7618ae577d9cd,2010,10180,10180,"Abilene, TX",48,TX,"Abilene, TX",00-0000,All Occupations,...,20.05,29.41,16170,18510,26800,41710,61170,NaN,NaN,NaN
1,6038df3058e7618ae577d9ce,2010,10420,10420,"Akron, OH",39,OH,"Akron, OH",00-0000,All Occupations,...,24.10,35.29,16930,20680,31760,50120,73410,NaN,NaN,NaN
2,6038df3058e7618ae577d9cf,2010,10500,10500,"Albany, GA",13,GA,"Albany, GA",00-0000,All Occupations,...,21.56,30.80,16060,18620,26970,44850,64060,NaN,NaN,NaN
3,6038df3058e7618ae577d9d0,2010,41420,10540,"Albany-Lebanon, OR",41,OR,"Salem, OR",00-0000,All Occupations,...,23.64,34.19,18920,22610,33200,49180,71120,NaN,NaN,NaN
4,6038df3058e7618ae577d9d1,2010,10580,10580,"Albany-Schenectady-Troy, NY",36,NY,"Albany-Schenectady-Troy, NY",00-0000,All Occupations,...,27.70,39.86,18630,24750,37780,57620,82900,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3835,6038df3058e7618ae577e8c8,2019,76750,38860,"Portland-South Portland, ME",23,76750,"Portland-South Portland, ME",00-0000,All Occupations,...,29.91,44.94,24230,29400,41160,62220,93470,NaN,NaN,NaN
3836,6038df3058e7618ae577e8c9,2019,77200,39300,"Providence-Warwick, RI-MA",44,77200,"Providence-Warwick, RI-MA",00-0000,All Occupations,...,33.20,49.06,24480,28940,42700,69050,102050,NaN,NaN,NaN
3837,6038df3058e7618ae577e8ca,2019,78100,44140,"Springfield, MA",25,78100,"Springfield, MA-CT",00-0000,All Occupations,...,31.19,45.62,25510,29910,42230,64870,94890,NaN,NaN,NaN
3838,6038df3058e7618ae577e8cb,2019,79600,49340,"Worcester, MA-CT",25,79600,"Worcester, MA-CT",00-0000,All Occupations,...,32.76,48.03,25630,30450,44000,68140,99900,NaN,NaN,NaN


In [70]:
df_total.columns

Index(['_id', 'YEAR', 'AREA', 'CBSA', 'MSA_Name', 'state_code', 'PRIM_STATE',
       'AREA_NAME', 'OCC_CODE', 'OCC_TITLE', 'GROUP', 'TOT_EMP', 'EMP_PRSE',
       'JOBS_1000', 'LOC QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE', 'H_PCT10',
       'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10', 'A_PCT25',
       'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'ANNUAL', 'HOURLY', 'OCC_GROUP'],
      dtype='object')

In [72]:
df_wage = df_total.drop(columns=['_id', 'AREA', 'MSA_Name', 'state_code', 'PRIM_STATE',
       'AREA_NAME', 'OCC_CODE', 'OCC_TITLE', 'GROUP', 'TOT_EMP', 'EMP_PRSE',
       'JOBS_1000', 'LOC QUOTIENT', 'H_MEAN', 'MEAN_PRSE', 'H_PCT10',
       'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10', 'A_PCT25',
       'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'ANNUAL', 'HOURLY', 'OCC_GROUP'])

In [73]:
def clean_raw_data(df_raw):
    # df_raw = df_raw[df_raw['month'] == 'December']
    # df_raw.drop(columns=['month', 'MSA', '_id'], inplace=True)
    # df_raw.drop(columns=['month', '_id'], inplace=True)
    df_raw.sort_values(by='CBSA', inplace=True)
    df_pivot = pd.pivot_table(df_raw, values = 'A_MEAN', index=['CBSA'], columns = 'YEAR').reset_index()
    df_pivot = df_pivot.rename_axis(None, axis=1)
    col_list = df_pivot.columns
    new_col_list = []
    for col in col_list:
        new_col_list.append(str(col))
    df_pivot.columns = new_col_list
    return df_pivot

In [75]:
df_meanwage = clean_raw_data(df_wage)

In [77]:
df_meanwage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   CBSA    384 non-null    int64
 1   2010    384 non-null    int64
 2   2011    384 non-null    int64
 3   2012    384 non-null    int64
 4   2013    384 non-null    int64
 5   2014    384 non-null    int64
 6   2015    384 non-null    int64
 7   2016    384 non-null    int64
 8   2017    384 non-null    int64
 9   2018    384 non-null    int64
 10  2019    384 non-null    int64
dtypes: int64(11)
memory usage: 33.1 KB


In [78]:
# export annual mean wage clean dataframe back to mongo
clean_collection = db.AnnualMeanWage_clean
df_dict = df_meanwage.to_dict(orient = 'records')
clean_collection.insert_many(df_dict)

In [79]:
## Now business applications

df_biz = pd.read_csv('../../../temp/CityStats/business_apps_grouped.csv')
df_biz

,CBSA Code,BA2010,BA2011,BA2012,BA2013,BA2014,BA2015,BA2016,BA2017,BA2018,BA2019
0,10180,774.0,818.0,929.0,839.0,978.0,1032.0,1039.0,1053.0,1093.0,1170.0
1,10420,4395.0,4610.0,4379.0,4327.0,4354.0,4639.0,4834.0,5175.0,5511.0,5787.0
2,10500,923.0,1033.0,1060.0,966.0,1107.0,1041.0,1161.0,1183.0,1300.0,1686.0
3,10540,564.0,544.0,537.0,509.0,631.0,608.0,687.0,744.0,843.0,831.0
4,10580,4825.0,5284.0,5150.0,5352.0,5216.0,5405.0,6050.0,7728.0,8960.0,7627.0
...,...,...,...,...,...,...,...,...,...,...,...
379,49420,924.0,1042.0,908.0,891.0,932.0,974.0,1038.0,1066.0,1213.0,1352.0
380,49620,2167.0,2151.0,2271.0,2123.0,2166.0,2232.0,2365.0,2561.0,2640.0,2756.0
381,49660,2652.0,2808.0,2982.0,2858.0,2890.0,2848.0,2850.0,3144.0,3319.0,3425.0
382,49700,655.0,648.0,614.0,690.0,656.0,715.0,666.0,782.0,892.0,1005.0


In [80]:
df_biz.columns

Index(['CBSA Code', 'BA2010', 'BA2011', 'BA2012', 'BA2013', 'BA2014', 'BA2015',
       'BA2016', 'BA2017', 'BA2018', 'BA2019'],
      dtype='object')

In [81]:
df_biz.columns = ['CBSA', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019']
df_biz

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,10180,774.0,818.0,929.0,839.0,978.0,1032.0,1039.0,1053.0,1093.0,1170.0
1,10420,4395.0,4610.0,4379.0,4327.0,4354.0,4639.0,4834.0,5175.0,5511.0,5787.0
2,10500,923.0,1033.0,1060.0,966.0,1107.0,1041.0,1161.0,1183.0,1300.0,1686.0
3,10540,564.0,544.0,537.0,509.0,631.0,608.0,687.0,744.0,843.0,831.0
4,10580,4825.0,5284.0,5150.0,5352.0,5216.0,5405.0,6050.0,7728.0,8960.0,7627.0
...,...,...,...,...,...,...,...,...,...,...,...
379,49420,924.0,1042.0,908.0,891.0,932.0,974.0,1038.0,1066.0,1213.0,1352.0
380,49620,2167.0,2151.0,2271.0,2123.0,2166.0,2232.0,2365.0,2561.0,2640.0,2756.0
381,49660,2652.0,2808.0,2982.0,2858.0,2890.0,2848.0,2850.0,3144.0,3319.0,3425.0
382,49700,655.0,648.0,614.0,690.0,656.0,715.0,666.0,782.0,892.0,1005.0


In [82]:
# export business apps clean dataframe back to mongo
clean_collection = db.BusinessApplications_clean
df_dict = df_biz.to_dict(orient = 'records')
clean_collection.insert_many(df_dict)

In [83]:
### Now the CPI

# Create instance of MongoClient
client = MongoClient()
# Connection URI
connStr = os.getenv('MONGO_CONN')
client = MongoClient(connStr)
# Select database
db = client['MSA']
# Select the collection within the database
data = db.CPI_RegionUrban_raw
# Convert entire collection to Pandas dataframe
df_total = pd.DataFrame(list(data.find()))
df_total

,_id,Region,series id,year,period,period_name,value
0,60375e722690f89598dfbb16,1,CUUR0100SA0,2019,M12,December,270.429
1,60375e722690f89598dfbb17,1,CUUR0100SA0,2019,M11,November,270.643
2,60375e722690f89598dfbb18,1,CUUR0100SA0,2019,M10,October,270.348
3,60375e722690f89598dfbb19,1,CUUR0100SA0,2019,M09,September,270.563
4,60375e722690f89598dfbb1a,1,CUUR0100SA0,2019,M08,August,270.548
...,...,...,...,...,...,...,...
475,60375e722690f89598dfbcf1,4,CUUR0400SA0,2010,M05,May,221.417
476,60375e722690f89598dfbcf2,4,CUUR0400SA0,2010,M04,April,221.202
477,60375e722690f89598dfbcf3,4,CUUR0400SA0,2010,M03,March,220.809
478,60375e722690f89598dfbcf4,4,CUUR0400SA0,2010,M02,February,220.179


In [85]:
df_raw = df_total[df_total['period_name'] == 'December']
df_raw.head()

,_id,Region,series id,year,period,period_name,value
0,60375e722690f89598dfbb16,1,CUUR0100SA0,2019,M12,December,270.429
12,60375e722690f89598dfbb22,1,CUUR0100SA0,2018,M12,December,265.286
24,60375e722690f89598dfbb2e,1,CUUR0100SA0,2017,M12,December,260.791
36,60375e722690f89598dfbb3a,1,CUUR0100SA0,2016,M12,December,256.427
48,60375e722690f89598dfbb46,1,CUUR0100SA0,2015,M12,December,251.670


In [86]:
# df_raw.drop(columns=['month', 'MSA', '_id'], inplace=True)
df_raw.drop(columns=['series id', '_id', 'period', 'period_name'], inplace=True)
df_raw.sort_values(by='Region', inplace=True)
df_raw

,Region,year,value
0,1,2019,270.429
12,1,2018,265.286
24,1,2017,260.791
36,1,2016,256.427
48,1,2015,251.670
60,1,2014,250.519
72,1,2013,249.567
84,1,2012,246.456
96,1,2011,241.987
108,1,2010,235.141


In [87]:
df_pivot = pd.pivot_table(df_raw, values = 'value', index=['Region'], columns = 'year').reset_index()
df_pivot = df_pivot.rename_axis(None, axis=1)
col_list = df_pivot.columns
new_col_list = []
for col in col_list:
    new_col_list.append(str(col))
df_pivot.columns = new_col_list

In [88]:
df_pivot

,Region,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,1,235.141,241.987,246.456,249.567,250.519,251.670,256.427,260.791,265.286,270.429
1,2,209.270,215.173,219.033,221.194,222.821,222.722,226.794,230.548,233.458,238.734
2,3,212.488,219.469,223.109,227.082,228.451,229.581,234.204,238.512,242.150,247.289
3,4,222.081,228.117,232.029,236.096,239.095,243.434,249.516,257.347,265.209,272.584


In [89]:
# export CPI clean dataframe back to mongo
clean_collection = db.CPI_RegionUrban_clean
df_dict = df_pivot.to_dict(orient = 'records')
clean_collection.insert_many(df_dict)

In [90]:
### Now we need to create a Labor Participation table

# Select the collection within the database
data = db.LaborForce_clean
# Convert entire collection to Pandas dataframe
df_labor = pd.DataFrame(list(data.find()))
df_labor

,_id,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,603dbd31680af8db371b1fb4,10180,76486,76723,76209,76347,75197,74411,74615,76071,77323,79499
1,603dbd31680af8db371b1fb5,10420,369203,361648,360436,356986,359933,358167,359554,358853,360974,357787
2,603dbd31680af8db371b1fb6,10500,70825,71135,69101,67391,65957,66377,68471,67478,67082,66999
3,603dbd31680af8db371b1fb7,10540,56691,56016,54190,53221,54075,55841,56853,57888,58764,58446
4,603dbd31680af8db371b1fb8,10580,449154,447519,453433,445629,438706,444496,441435,440205,444430,449737
...,...,...,...,...,...,...,...,...,...,...,...,...
379,603dbd31680af8db371b212f,49420,114826,115596,114403,112408,115915,116743,119089,121256,124890,128962
380,603dbd31680af8db371b2130,49620,229344,231145,232525,228768,229489,232244,233206,232871,234409,237714
381,603dbd31680af8db371b2131,49660,265800,261511,259693,254507,251949,253137,245454,241154,240643,236372
382,603dbd31680af8db371b2132,49700,74371,74220,72725,71203,70521,71757,71897,72796,72979,77036


In [94]:
# Select the collection within the database
data = db.Population_clean
# Convert entire collection to Pandas dataframe
df_pop = pd.DataFrame(list(data.find()))
df_pop

,_id,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,603dc65f680af8db371b2438,11540,225948,227548,228656,229978,231627,233146,234200,235811,237011,237974
1,603dc65f680af8db371b2439,12020,193439,194375,195984,197590,198361,202565,205402,209106,211468,213750
2,603dc65f680af8db371b243a,14010,169810,170909,172721,175037,174105,173141,173070,172754,172410,171517
3,603dc65f680af8db371b243b,15540,211538,213235,213962,214827,215676,217152,217447,218975,219939,220411
4,603dc65f680af8db371b243c,19660,590506,591926,594969,600236,608604,621166,635859,647867,658186,668365
...,...,...,...,...,...,...,...,...,...,...,...,...
379,603dc65f680af8db371b25b3,38340,131317,130547,130284,129488,128856,127858,127067,126353,125901,124944
380,603dc65f680af8db371b25b4,41700,2153009,2193923,2236981,2280002,2328894,2379232,2426276,2472121,2512379,2550960
381,603dc65f680af8db371b25b5,42100,263147,264858,266340,268816,270861,273503,274396,274856,273841,273213
382,603dc65f680af8db371b25b6,44180,437349,440432,444329,448427,451556,455593,457917,462090,466273,470300


In [95]:
df_pop = df_pop.sort_values(by='CBSA').reset_index(drop=True)
df_pop

,_id,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,603dc65f680af8db371b2522,10180,165585,166634,167442,167473,168342,169688,170017,170429,171150,172060
1,603dc65f680af8db371b2541,10420,703031,703200,702109,703621,704908,704382,703524,703987,703855,703479
2,603dc65f680af8db371b2469,10500,154145,154545,153976,152667,151949,150387,149137,148090,147840,146726
3,603dc65f680af8db371b2583,10540,116891,118164,118273,118405,119042,120236,122769,125035,127451,129749
4,603dc65f680af8db371b2523,10580,871082,872778,874698,877065,878113,879085,879792,882158,882263,880381
...,...,...,...,...,...,...,...,...,...,...,...,...
379,603dc65f680af8db371b2581,49420,244256,245926,246127,246485,246870,247800,249327,249922,250562,250873
380,603dc65f680af8db371b256e,49620,435413,436701,437466,438932,440475,441741,443693,445484,447847,449058
381,603dc65f680af8db371b24a5,49660,564826,562347,558530,555850,553039,548798,544582,541545,538226,536081
382,603dc65f680af8db371b259e,49700,167099,166971,166963,167597,168311,169327,170694,172782,173905,175639


In [96]:
df_labor.columns

Index(['_id', 'CBSA', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019'],
      dtype='object')

In [102]:
years = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
df_part = pd.DataFrame()
df_part['CBSA'] = df_labor['CBSA']
for year in years:
    df_part[year] = df_labor[year]/df_pop[year]
df_part

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,10180,0.461914,0.460428,0.455137,0.455876,0.446692,0.438517,0.438868,0.446350,0.451785,0.462042
1,10420,0.525159,0.514289,0.513362,0.507356,0.510610,0.508484,0.511076,0.509744,0.512853,0.508597
2,10500,0.459470,0.460287,0.448778,0.441425,0.434073,0.441375,0.459115,0.455655,0.453747,0.456627
3,10540,0.484990,0.474053,0.458177,0.449483,0.454251,0.464428,0.463089,0.462974,0.461071,0.450454
4,10580,0.515628,0.512752,0.518388,0.508091,0.499601,0.505635,0.501749,0.499009,0.503739,0.510844
...,...,...,...,...,...,...,...,...,...,...,...
379,49420,0.470105,0.470044,0.464813,0.456044,0.469539,0.471118,0.477642,0.485175,0.498440,0.514053
380,49620,0.526727,0.529298,0.531527,0.521192,0.521003,0.525747,0.525602,0.522737,0.523413,0.529361
381,49660,0.470587,0.465035,0.464958,0.457870,0.455572,0.461257,0.450720,0.445307,0.447104,0.440926
382,49700,0.445071,0.444508,0.435576,0.424847,0.418992,0.423778,0.421204,0.421317,0.419649,0.438604


In [103]:
# export labor participation dataframe back to mongo
clean_collection = db.LaborParticipation_clean
df_dict = df_part.to_dict(orient = 'records')
clean_collection.insert_many(df_dict)